# **FASE 1: Clusterización**

**1. IMPORTACIÓN DE LAS LIBRERÍAS NECESARIAS PARA ESTA SEGUNDA FASE**

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import os
import csv

import umap
import hdbscan

%matplotlib inline

**2. CARGA DE LOS DATOS**

A continuación se declarará el siguiente dataframe generado en pandas y cargado a través del archivo CSV generado en la *FASE 0** llamado **datasetFinal_fifa2019** en la ruta relativa "'../../data/datasetFinal_fifa2019.csv'", donde se encuentra el dataset final para proceder a la clusterización. 

In [2]:
df_ = pd.read_csv('../../data/datasetFinal_fifa2019.csv')

**3. INGENIERIA DE ATRIBUTOS**

Previamente se han realizado varias pruebas donde se ha descartado entre otras acciones la codificación de variables categóricas mediante la función *get_dummies*.

A continuación, se describirá y llevará a cabo la metodología que ha proporcionado la clusterización más optimizada en base a los resultados obtenidos.

Como primer paso y según lo requerido eliminaremos las variables categóricas.

In [3]:
df = df_.drop(['ID',
 'Name',
 'Nationality',
 'Club',
 'Preferred Foot'], axis=1)

Como siguiente paso se hará un reescalado de las variables puesto que tiene distintos órdenes de magnitud y para ello se calculará el logaritmo neperiano.

In [4]:
df_log = np.log(df)

Una vez hecho esto se hará una primera reducción del dataset obteniendo la media de todas aquellas variables que en este caso tenían las mismas características y pertenecían en conjunto a las estadísticas de juego de cada registro.

In [5]:
df_log['avg_stats'] = df_log[[ 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes','Work']].mean(axis=1)

En lugar de eliminar las columnas anteriores, se generará un nuevo dataset donde se incluya la columna que represente la media y esas queden fuera.

In [6]:
df_avg = df_log[['Overall', 'Potential','Weight','International Reputation','Special','avg_stats']]

**4. REDUCCIÓN DEL DATASET**

En esta ocasión y de una manera más técnica se le aplicará el método **UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction**, un algoritmo que según las investigaciones y las pruebas realizadas previamente, trabaja de manera muy eficiente con los modelos de clusterización.

In [7]:
#Es necesario convertir el dataframe en array para que UMAP pueda procesarlo.
fifa = df_avg.to_numpy()

In [30]:
reducer = umap.UMAP(random_state=42)
reducer.fit(fifa.data)

Como resultado se obtendrá un matriz reducida llamada **embedding** con la que se trabajará a partir de ahora.

In [9]:
embedding = reducer.transform(fifa.data)

**5. APLICACIÓN DE LOS DISTINTOS MODELOS DE CLUSTERIZACIÓN**

A continuación, serán aplicados los distintos modelos seleccionados previamente al **embedding**.

In [31]:
plt.scatter(embedding[:, 0], embedding[:, 1], cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(11)-0.5).set_ticks(np.arange(10))
plt.title('UMAP projection of the Digits dataset', fontsize=24);

**KMEANS**

In [32]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=6, random_state=42).fit(fifa.data)

kmeans

In [33]:
kmeans.predict(fifa.data)

In [34]:
kmeans.cluster_centers_[0]

In [35]:
kmeans = KMeans(n_clusters=5, random_state=42).fit(embedding)

kmeans

In [36]:
plt.scatter(embedding[:, 0], embedding[:, 1], s=5)
plt.gca().set_aspect('equal', 'datalim')

In [37]:
plt.scatter(embedding[:, 0], embedding[:, 1], c=kmeans.labels_, cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(kmeans.n_clusters+1)-0.5).set_ticks(np.arange(kmeans.n_clusters))
plt.title('K-Means over the Digits dataset', fontsize=20);

**DBSCAN**

In [38]:
from sklearn.cluster import DBSCAN

clustering = DBSCAN().fit(embedding)

clustering

In [39]:
plt.scatter(embedding[:, 0], embedding[:, 1], c=clustering.labels_, cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(len(np.unique(clustering.labels_))+1)-0.5).set_ticks(np.arange(len(np.unique(clustering.labels_))))
plt.title('DBSCAN over the Digits dataset', fontsize=24);

**HDBSCAN**

In [40]:
clusterer = hdbscan.HDBSCAN()

clusterer.fit(embedding)

clusterer

In [41]:
plt.scatter(embedding[:, 0], embedding[:, 1], c=clusterer.labels_, cmap='Spectral', s=5)
plt.gca().set_aspect('equal', 'datalim')
plt.colorbar(boundaries=np.arange(len(np.unique(clusterer.labels_)))+0.5).set_ticks(np.arange(len(np.unique(clusterer.labels_))))
plt.title('HDBSCAN over the Digits dataset', fontsize=24);

**6. PRIMERAS CONCLUSIONES**

Se puede observar una agrupación bastante limpia de los jugadores donde en concreto el modelo **DBSCAN** ha logrado darle una clusterización muy efectiva a la hora de etiquetar los colores en la representación gráfica. 

Para saber mas a cerca de este modelo: https://es.wikipedia.org/wiki/DBSCAN

**7. ETIQUETADO**

Finalmente se generará una columna nueva al dataframe inicial que clasifique a cada jugador en su clúster corresponiente, para ello se volverán a llamar a las etiquetas producidas por el modelo. Esto se llevará a cabo asignando a esta nueva columna el elemento **clustering.labels_** perteneciente en este caso al producido por el modelo **DBSCAN**.

In [ ]:
df_['cluster'] = clustering.labels_

**BONUS**

Para la posterior visualización de la información se recuperará la columna donde se contiene la media de las estadísticas de juego.

In [29]:
df_['avg_stats'] = df_[[ 'Crossing',
 'Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Acceleration',
 'SprintSpeed',
 'Agility',
 'Reactions',
 'Balance',
 'ShotPower',
 'Jumping',
 'Stamina',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Penalties',
 'Composure',
 'Marking',
 'StandingTackle',
 'SlidingTackle',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes']].mean(axis=1)

**8. EXPORTACIÓN DE LOS DATOS**

Finalmente los datos serán exportados en un csv con el nombre de **4_clusteres_fifa2019_stats** a la carpeta *data* que se encuentra en el repositorio del proyecto. Ruta relativa "../../data".

In [27]:
def exportación (df):
    name_file = '4_clusteres_fifa2019_stats.csv'
    path = '../../data'
    output = os.path.join(path, name_file)
    df_.to_csv(output, index = False)

In [28]:
exportación (df_)